In [1]:
import polars as pl
import plotly.express as px
import pandas as pd

df = pl.read_csv("aao.csv")
df= df.drop(["MODE_TAG", "modified_at", "modified_by"])
print(df)

shape: (18_752, 4)
┌────────────┬─────────────┬────────────────┬────────────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT        │
│ ---        ┆ ---         ┆ ---            ┆ ---        │
│ str        ┆ str         ┆ str            ┆ f64        │
╞════════════╪═════════════╪════════════════╪════════════╡
│ 2000-03-01 ┆ NDB         ┆ INTLBOND       ┆ 1.0000e-12 │
│ 2000-03-01 ┆ NDB         ┆ FRN            ┆ 0.043348   │
│ 2000-03-01 ┆ NDB         ┆ TBILL          ┆ 0.0        │
│ 2000-03-01 ┆ NDGI        ┆ USLG           ┆ 0.251856   │
│ 2000-03-01 ┆ NDGI        ┆ USSM           ┆ 0.059764   │
│ …          ┆ …           ┆ …              ┆ …          │
│ 2000-03-01 ┆ NDB         ┆ MGDFUT         ┆ 0.080686   │
│ 2000-03-01 ┆ NDB         ┆ AGG            ┆ 0.371721   │
│ 2000-03-01 ┆ NDB         ┆ LTT            ┆ 0.0        │
│ 2000-03-01 ┆ NDB         ┆ TIPS           ┆ 0.0        │
│ 2000-03-01 ┆ NDB         ┆ HY             ┆ 0.0        │
└────────────┴─────────────┴─────────

In [10]:
mean = df.group_by(["STRATEGY_ID","ASSET_CLASS_ID"]).agg(pl.col("WGT").mean())
print(mean)
mean = mean.to_pandas()
fig = px.bar(mean, x='STRATEGY_ID', y='WGT', color='ASSET_CLASS_ID', title="Mean Target Allocation")
fig.write_html("mean.html")
fig
fig2 = px.bar(mean, x='ASSET_CLASS_ID', y='WGT', color='STRATEGY_ID', title="Mean Target Allocation")

shape: (64, 3)
┌─────────────┬────────────────┬────────────┐
│ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT        │
│ ---         ┆ ---            ┆ ---        │
│ str         ┆ str            ┆ f64        │
╞═════════════╪════════════════╪════════════╡
│ NDCB        ┆ AGG            ┆ 0.479307   │
│ NDLTG       ┆ FRN            ┆ 0.003458   │
│ NDLTG       ┆ ABSRET         ┆ 5.4608e-18 │
│ NDGI        ┆ LTT            ┆ 0.05905    │
│ NDB         ┆ COMM           ┆ 0.001065   │
│ …           ┆ …              ┆ …          │
│ NDB         ┆ USSM           ┆ 0.068735   │
│ NDLTG       ┆ USLG           ┆ 0.320246   │
│ NDB         ┆ INTLSM         ┆ 0.003209   │
│ NDGI        ┆ USSM           ┆ 0.083714   │
│ NDGI        ┆ INTLLG         ┆ 0.1517     │
└─────────────┴────────────────┴────────────┘


In [12]:
std = df.group_by(["STRATEGY_ID","ASSET_CLASS_ID"]).agg(pl.col("WGT").std())
print(std)
std = std.to_pandas()
fig = px.bar(std, x='STRATEGY_ID', y='WGT', color='ASSET_CLASS_ID', title="Standard Deviation of Monthly Target Allocations 2000-2024")
fig.write_html("std.html")
fig

shape: (64, 3)
┌─────────────┬────────────────┬────────────┐
│ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT        │
│ ---         ┆ ---            ┆ ---        │
│ str         ┆ str            ┆ f64        │
╞═════════════╪════════════════╪════════════╡
│ NDB         ┆ INTLSM         ┆ 0.006818   │
│ NDB         ┆ INTLBOND       ┆ 4.6352e-13 │
│ NDCB        ┆ REIT           ┆ 0.051386   │
│ NDGI        ┆ USSM           ┆ 0.024638   │
│ NDLTG       ┆ LTT            ┆ 0.021747   │
│ …           ┆ …              ┆ …          │
│ NDLTG       ┆ INTLSM         ┆ 0.026897   │
│ NDCB        ┆ USSM           ┆ 0.020336   │
│ NDB         ┆ FRN            ┆ 0.03195    │
│ NDLTG       ┆ REIT           ┆ 0.061051   │
│ NDCB        ┆ EM             ┆ 0.020522   │
└─────────────┴────────────────┴────────────┘


In [22]:
us = df.filter(pl.col("ASSET_CLASS_ID").is_in(["USLG","USSM"]))
summed = us.group_by(["STRATEGY_ID", "DATE"]).agg(pl.col("WGT").sum())
us = us.join(summed, on=["STRATEGY_ID", "DATE"],
                how="inner",)
us = us.with_columns((pl.col("WGT") / pl.col("WGT_right")).alias("split"))
us_mean = us.group_by(["STRATEGY_ID", "ASSET_CLASS_ID"]).agg(pl.col("split").mean())
print(us_mean)
to_plot = us_mean.to_pandas()
fig = px.bar(to_plot, x='STRATEGY_ID', y='split', color='ASSET_CLASS_ID', title="Mean US Large and Small Cap Allocation")
fig.write_html("us.html")
fig

shape: (8, 3)
┌─────────────┬────────────────┬──────────┐
│ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ split    │
│ ---         ┆ ---            ┆ ---      │
│ str         ┆ str            ┆ f64      │
╞═════════════╪════════════════╪══════════╡
│ NDLTG       ┆ USLG           ┆ 0.689097 │
│ NDLTG       ┆ USSM           ┆ 0.310903 │
│ NDCB        ┆ USSM           ┆ 0.337554 │
│ NDGI        ┆ USSM           ┆ 0.20733  │
│ NDCB        ┆ USLG           ┆ 0.662446 │
│ NDB         ┆ USLG           ┆ 0.784402 │
│ NDB         ┆ USSM           ┆ 0.215598 │
│ NDGI        ┆ USLG           ┆ 0.79267  │
└─────────────┴────────────────┴──────────┘


In [30]:
ndb = us.filter(pl.col("STRATEGY_ID").is_in(["NDB"]))
to_plot = ndb.to_pandas()
fig = px.bar(to_plot, x='DATE', y='split', color='ASSET_CLASS_ID', title="Mean US Large and Small Cap Allocation")
fig

In [55]:
min = us.filter(pl.col("ASSET_CLASS_ID").is_in(["USLG"]))
min = min.group_by(["DATE"]).agg(pl.col("split").min())

max = us.filter(pl.col("ASSET_CLASS_ID").is_in(["USLG"]))
max = max.group_by(["DATE"]).agg(pl.col("split").max())

diff = min.join(max, on=["DATE"], how="inner")
diff = diff.with_columns((pl.col("split_right") - pl.col("split")).alias("diff"))
diff.sort("DATE")
diff = diff.to_pandas()
fig = px.bar(diff, x='DATE', y='diff', title="USLG / USSM Allocation Range")   
fig

In [23]:
us_std = us.group_by(["STRATEGY_ID","ASSET_CLASS_ID"]).agg(pl.col("split").std())
print(us_std)
us_std = us_std.to_pandas()
fig = px.bar(us_std, x='STRATEGY_ID', y='split', color='ASSET_CLASS_ID', title="Standard Deviation of US Large and Small Cap Allocation")
fig.write_html("us_std.html")
fig

shape: (8, 3)
┌─────────────┬────────────────┬──────────┐
│ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ split    │
│ ---         ┆ ---            ┆ ---      │
│ str         ┆ str            ┆ f64      │
╞═════════════╪════════════════╪══════════╡
│ NDGI        ┆ USSM           ┆ 0.04403  │
│ NDLTG       ┆ USSM           ┆ 0.067454 │
│ NDB         ┆ USLG           ┆ 0.043092 │
│ NDCB        ┆ USLG           ┆ 0.067373 │
│ NDLTG       ┆ USLG           ┆ 0.067454 │
│ NDCB        ┆ USSM           ┆ 0.067373 │
│ NDGI        ┆ USLG           ┆ 0.04403  │
│ NDB         ┆ USSM           ┆ 0.043092 │
└─────────────┴────────────────┴──────────┘


In [40]:
df.filter(pl.col("DATE").is_in(["2024-05-01"])).filter(pl.col("ASSET_CLASS_ID").is_in(["USLG","USSM"]))

DATE,STRATEGY_ID,ASSET_CLASS_ID,WGT
str,str,str,f64
"""2024-05-01""","""NDCB""","""USLG""",0.099549
"""2024-05-01""","""NDCB""","""USSM""",0.084868
"""2024-05-01""","""NDB""","""USLG""",0.198896
"""2024-05-01""","""NDB""","""USSM""",0.094842
"""2024-05-01""","""NDGI""","""USLG""",0.262292
"""2024-05-01""","""NDGI""","""USSM""",0.114209
"""2024-05-01""","""NDLTG""","""USLG""",0.263341
"""2024-05-01""","""NDLTG""","""USSM""",0.20048


In [69]:
fixed_list = ["FRN", "LTT", "TBILL", "AGG", "HY"]
fixed = df.filter(pl.col("ASSET_CLASS_ID").is_in(fixed_list))

summed = fixed.group_by(["STRATEGY_ID", "DATE"]).agg(pl.col("WGT").sum())
fixed = fixed.join(summed, on=["STRATEGY_ID", "DATE"],
                how="inner",)
fixed = fixed.with_columns((pl.col("WGT") / pl.col("WGT_right")).alias("split"))
print(fixed.filter(pl.col("STRATEGY_ID").is_in(["NDB"])).filter(pl.col("DATE").is_in(["2022-08-01"])))
fixed_mean = fixed.group_by(["STRATEGY_ID", "ASSET_CLASS_ID"]).agg(pl.col("split").mean())
print(fixed_mean)
to_plot = fixed_mean.to_pandas()
fig = px.bar(to_plot, x='STRATEGY_ID', y='split', color='ASSET_CLASS_ID', title="Mean Fixed Income Allocation")
fig.write_html("fixed.html")
fig

shape: (5, 6)
┌────────────┬─────────────┬────────────────┬──────────┬───────────┬──────────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT      ┆ WGT_right ┆ split    │
│ ---        ┆ ---         ┆ ---            ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ str         ┆ str            ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═════════════╪════════════════╪══════════╪═══════════╪══════════╡
│ 2022-08-01 ┆ NDB         ┆ AGG            ┆ 0.251263 ┆ 0.40222   ┆ 0.624692 │
│ 2022-08-01 ┆ NDB         ┆ LTT            ┆ 0.032002 ┆ 0.40222   ┆ 0.079563 │
│ 2022-08-01 ┆ NDB         ┆ HY             ┆ 0.0      ┆ 0.40222   ┆ 0.0      │
│ 2022-08-01 ┆ NDB         ┆ FRN            ┆ 0.066404 ┆ 0.40222   ┆ 0.165094 │
│ 2022-08-01 ┆ NDB         ┆ TBILL          ┆ 0.05255  ┆ 0.40222   ┆ 0.130651 │
└────────────┴─────────────┴────────────────┴──────────┴───────────┴──────────┘
shape: (20, 3)
┌─────────────┬────────────────┬──────────┐
│ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ split    │
│ -

In [71]:
for asset in fixed_list:
    min = fixed.filter(pl.col("ASSET_CLASS_ID").is_in([asset]))
    min = min.group_by(["DATE"]).agg(pl.col("split").min())

    max = fixed.filter(pl.col("ASSET_CLASS_ID").is_in([asset]))
    max = max.group_by(["DATE"]).agg(pl.col("split").max())
    print(max.sort("DATE"))

    diff = min.join(max, on=["DATE"], how="inner")
    print(diff.sort("DATE"))
    diff = diff.with_columns((pl.col("split_right") - pl.col("split")).alias("diff"))
    diff.sort("DATE")
    diff = diff.to_pandas()
    fig = px.bar(diff, x='DATE', y='diff', title=asset + " Allocation Range")   
    fig.show()

shape: (1_172, 6)
┌────────────┬─────────────┬────────────────┬──────────┬───────────┬──────────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT      ┆ WGT_right ┆ split    │
│ ---        ┆ ---         ┆ ---            ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ str         ┆ str            ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═════════════╪════════════════╪══════════╪═══════════╪══════════╡
│ 2000-01-01 ┆ NDCB        ┆ FRN            ┆ 0.053082 ┆ 0.593194  ┆ 0.089486 │
│ 2000-01-01 ┆ NDB         ┆ FRN            ┆ 0.018675 ┆ 0.403862  ┆ 0.04624  │
│ 2000-01-01 ┆ NDGI        ┆ FRN            ┆ 0.0      ┆ 0.22792   ┆ 0.0      │
│ 2000-01-01 ┆ NDLTG       ┆ FRN            ┆ 0.0      ┆ 0.125777  ┆ 0.0      │
│ 2000-02-01 ┆ NDCB        ┆ FRN            ┆ 0.070411 ┆ 0.59792   ┆ 0.11776  │
│ …          ┆ …           ┆ …              ┆ …        ┆ …         ┆ …        │
│ 2024-04-01 ┆ NDLTG       ┆ FRN            ┆ 0.0      ┆ 0.091025  ┆ 0.0      │
│ 2024-05-01 ┆ NDCB   

shape: (1_172, 6)
┌────────────┬─────────────┬────────────────┬──────────┬───────────┬──────────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT      ┆ WGT_right ┆ split    │
│ ---        ┆ ---         ┆ ---            ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ str         ┆ str            ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═════════════╪════════════════╪══════════╪═══════════╪══════════╡
│ 2000-01-01 ┆ NDCB        ┆ LTT            ┆ 0.011958 ┆ 0.593194  ┆ 0.020159 │
│ 2000-01-01 ┆ NDB         ┆ LTT            ┆ 0.02299  ┆ 0.403862  ┆ 0.056925 │
│ 2000-01-01 ┆ NDGI        ┆ LTT            ┆ 0.019908 ┆ 0.22792   ┆ 0.087346 │
│ 2000-01-01 ┆ NDLTG       ┆ LTT            ┆ 0.019151 ┆ 0.125777  ┆ 0.152264 │
│ 2000-02-01 ┆ NDCB        ┆ LTT            ┆ 0.0      ┆ 0.59792   ┆ 0.0      │
│ …          ┆ …           ┆ …              ┆ …        ┆ …         ┆ …        │
│ 2024-04-01 ┆ NDLTG       ┆ LTT            ┆ 0.027565 ┆ 0.091025  ┆ 0.302834 │
│ 2024-05-01 ┆ NDCB   

shape: (1_172, 6)
┌────────────┬─────────────┬────────────────┬──────────┬───────────┬──────────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT      ┆ WGT_right ┆ split    │
│ ---        ┆ ---         ┆ ---            ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ str         ┆ str            ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═════════════╪════════════════╪══════════╪═══════════╪══════════╡
│ 2000-01-01 ┆ NDCB        ┆ TBILL          ┆ 0.018852 ┆ 0.593194  ┆ 0.031781 │
│ 2000-01-01 ┆ NDB         ┆ TBILL          ┆ 0.0      ┆ 0.403862  ┆ 0.0      │
│ 2000-01-01 ┆ NDGI        ┆ TBILL          ┆ 0.0      ┆ 0.22792   ┆ 0.0      │
│ 2000-01-01 ┆ NDLTG       ┆ TBILL          ┆ 0.0      ┆ 0.125777  ┆ 0.0      │
│ 2000-02-01 ┆ NDCB        ┆ TBILL          ┆ 0.0      ┆ 0.59792   ┆ 0.0      │
│ …          ┆ …           ┆ …              ┆ …        ┆ …         ┆ …        │
│ 2024-04-01 ┆ NDLTG       ┆ TBILL          ┆ 0.063459 ┆ 0.091025  ┆ 0.697166 │
│ 2024-05-01 ┆ NDCB   

shape: (1_172, 6)
┌────────────┬─────────────┬────────────────┬──────────┬───────────┬──────────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT      ┆ WGT_right ┆ split    │
│ ---        ┆ ---         ┆ ---            ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ str         ┆ str            ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪═════════════╪════════════════╪══════════╪═══════════╪══════════╡
│ 2000-01-01 ┆ NDCB        ┆ AGG            ┆ 0.509302 ┆ 0.593194  ┆ 0.858575 │
│ 2000-01-01 ┆ NDB         ┆ AGG            ┆ 0.362198 ┆ 0.403862  ┆ 0.896835 │
│ 2000-01-01 ┆ NDGI        ┆ AGG            ┆ 0.208012 ┆ 0.22792   ┆ 0.912654 │
│ 2000-01-01 ┆ NDLTG       ┆ AGG            ┆ 0.106626 ┆ 0.125777  ┆ 0.847736 │
│ 2000-02-01 ┆ NDCB        ┆ AGG            ┆ 0.527509 ┆ 0.59792   ┆ 0.88224  │
│ …          ┆ …           ┆ …              ┆ …        ┆ …         ┆ …        │
│ 2024-04-01 ┆ NDLTG       ┆ AGG            ┆ 0.0      ┆ 0.091025  ┆ 0.0      │
│ 2024-05-01 ┆ NDCB   

shape: (1_172, 6)
┌────────────┬─────────────┬────────────────┬─────┬───────────┬───────┐
│ DATE       ┆ STRATEGY_ID ┆ ASSET_CLASS_ID ┆ WGT ┆ WGT_right ┆ split │
│ ---        ┆ ---         ┆ ---            ┆ --- ┆ ---       ┆ ---   │
│ str        ┆ str         ┆ str            ┆ f64 ┆ f64       ┆ f64   │
╞════════════╪═════════════╪════════════════╪═════╪═══════════╪═══════╡
│ 2000-01-01 ┆ NDCB        ┆ HY             ┆ 0.0 ┆ 0.593194  ┆ 0.0   │
│ 2000-01-01 ┆ NDB         ┆ HY             ┆ 0.0 ┆ 0.403862  ┆ 0.0   │
│ 2000-01-01 ┆ NDGI        ┆ HY             ┆ 0.0 ┆ 0.22792   ┆ 0.0   │
│ 2000-01-01 ┆ NDLTG       ┆ HY             ┆ 0.0 ┆ 0.125777  ┆ 0.0   │
│ 2000-02-01 ┆ NDCB        ┆ HY             ┆ 0.0 ┆ 0.59792   ┆ 0.0   │
│ …          ┆ …           ┆ …              ┆ …   ┆ …         ┆ …     │
│ 2024-04-01 ┆ NDLTG       ┆ HY             ┆ 0.0 ┆ 0.091025  ┆ 0.0   │
│ 2024-05-01 ┆ NDCB        ┆ HY             ┆ 0.0 ┆ 0.631158  ┆ 0.0   │
│ 2024-05-01 ┆ NDB         ┆ HY             ┆ 